In [1]:
import os
import numpy as np
import cv2
import matplotlib
from matplotlib import image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Conv1D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras import models
from tensorflow.keras import utils

2023-10-18 19:19:20.653374: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-18 19:19:20.916842: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-18 19:19:20.916896: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-18 19:19:20.918615: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-18 19:19:21.056807: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-18 19:19:21.058650: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
# circle, kite, parallelogram, rectangle, rhombus, square, trapezoid, triangle
labels = ['circle', 'kite', 'parallelogram', 'rectangle', 'rhombus', 'square', 'trapezoid', 'triangle']
image_shape = (224, 224, 1)

In [3]:
def make_labels(directory, data=[], y_hat=[], label=0):
    for root, dirs, files in os.walk(directory):
        for file in files:
            img = np.reshape(cv2.cvtColor(matplotlib.image.imread(directory+file), cv2.COLOR_RGB2GRAY),image_shape)
            data.append(img)
        y_hat = [label] * len(data)
    return np.array(data), np.array(y_hat)

In [4]:
circle, y_circle = [], []
circle, y_circle = make_labels('./../../shape-train/circle/', data=circle, y_hat=y_circle, label=labels.index('circle'))
print("circle loaded")

kite, y_kite = [], []
kite, y_kite = make_labels('./../../shape-train/kite/', data=kite, y_hat=y_kite, label=labels.index('kite'))
print("kite loaded")

parallelogram, y_parallelogram = [], []
parallelogram, y_parallelogram = make_labels('./../../shape-train/parallelogram/', data=parallelogram, y_hat=y_parallelogram, label=labels.index('parallelogram'))
print("parallelogram loaded")

rectangle, y_rectangle = [], []
rectangle, y_rectangle = make_labels('./../../shape-train/rectangle/', data=rectangle, y_hat=y_rectangle, label=labels.index('rectangle'))
print("rectangle loaded")

rhombus, y_rhombus = [], []
rhombus, y_rhombus = make_labels('./../../shape-train/rhombus/', data=rhombus, y_hat=y_rhombus, label=labels.index('rhombus'))
print("rhombus loaded")

square, y_square = [], []
square, y_square = make_labels('./../../shape-train/square/', data=square, y_hat=y_square, label=labels.index('square'))
print("square loaded")

trapezoid, y_trapezoid = [], []
trapezoid, y_trapezoid = make_labels('./../../shape-train/trapezoid/', data=trapezoid, y_hat=y_trapezoid, label=labels.index('trapezoid'))
print("trapezoid loaded")

triangle, y_triangle = [], []
triangle, y_triangle = make_labels('./../../shape-train/triangle/', data=triangle, y_hat=y_triangle, label=labels.index('triangle'))
print("triangle loaded")

circle loaded
kite loaded
parallelogram loaded
rectangle loaded
rhombus loaded
square loaded
trapezoid loaded
triangle loaded


In [5]:
X, y = np.vstack((circle, kite, parallelogram, rectangle, rhombus, square, trapezoid, triangle)), np.hstack((y_circle, y_kite, y_parallelogram, y_rectangle, y_rhombus, y_square, y_trapezoid, y_triangle)).reshape(-1, 1)

In [6]:
classifier = Sequential()
# Convolution layer, with 32 random 3x3 kernels
c1 = Conv2D(48, (3,3), padding='same', input_shape = image_shape, activation = 'relu')
classifier.add(c1)
classifier.add(MaxPooling2D(pool_size=(3, 3)))
classifier.add(Dropout(0.5))
classifier.add(Flatten()) 
w1 = Dense(units = 32, activation = 'relu')
w2 = Dense(units = 16, activation = 'relu')
w3 = Dense(units = 4, activation = 'relu')
w_end = Dense(units = 3, activation = 'softmax')
classifier.add(w1)
classifier.add(Dropout(0.5)) 
classifier.add(w2)
classifier.add(Dropout(0.5)) 
classifier.add(w_end)
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 48)      480       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 48)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 74, 74, 48)        0         
                                                                 
 flatten (Flatten)           (None, 262848)            0         
                                                                 
 dense (Dense)               (None, 32)                8411168   
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                        

In [7]:
classifier.compile( optimizer = 'rmsprop',
                    loss = 'categorical_crossentropy', 
                    metrics = ['accuracy'])

In [8]:
y_cat = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=.2, random_state=42)

print(X_train.shape)
print(y_train.shape)

(16000, 224, 224, 1)
(16000, 8)


In [9]:
history = classifier.fit(X_train, y_train, batch_size=40, epochs=10, verbose=1, validation_data=(X_test, y_test))

: 